### Carrega as bibliotecas necessárias para o projeto

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import tensorflow as tf
from tensorflow import keras

from google.colab import drive

import zipfile
import tarfile
import os
import shutil

from datetime import datetime

warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Mounted at /content/drive


### Faz o download dos arquivos

In [2]:
# Definição do caminho dos diretórios
compressed_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed'
images_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Images'

# Renomeia o diretório Compressed se já existir
if os.path.exists(compressed_folder):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    novo_dir = compressed_folder + f"_backup_{timestamp}"
    shutil.move(compressed_folder, novo_dir)
    print(f"Diretório 'Compressed' existente foi renomeado para: {novo_dir}")

# Criação dos diretórios
os.makedirs(compressed_folder, exist_ok=True)
os.makedirs(images_folder, exist_ok=True)

# Define os a url e os arquivos a serem baixados
url_base = 'https://zenodo.org/record/10219797/files/'
files_to_download = ['macroscopic0.zip', 'macroscopic1.tar.gz',
                     'macroscopic2.tar.gz', 'macroscopic3.tar.gz',
                     'macroscopic4.tar.gz']

# Faz o download dos arquivos
for arquivo in files_to_download:
    url = url_base + arquivo
    destino = os.path.join(compressed_folder, arquivo)

    print(f'Baixando o arquivo {arquivo}...')
    !wget -O "{destino}" "{url}"
    print(f'Download do arquivo {arquivo} finalizado.\n')

print('Todos os arquivos baixados com sucesso !')


Baixando o arquivo macroscopic0.zip...
--2025-02-19 21:51:32--  https://zenodo.org/record/10219797/files/macroscopic0.zip
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.48.194, 188.185.43.25, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/10219797/files/macroscopic0.zip [following]
--2025-02-19 21:51:32--  https://zenodo.org/records/10219797/files/macroscopic0.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1637314819 (1.5G) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed/macroscopic0.zip’

/content/drive/MyDr 100%[===================>]   1.52G  16.2MB/s    in 92s     

2025-02-19 21:53:04 (17.0 MB/s) - ‘/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed/macroscopic0.zip’ saved [1637314819/1637314819]

Download do arquivo macroscopic0.zip finaliz

### Processando arquivos Zip

In [3]:
def verifica_arquivos_zip(arquivo_zip):
  source_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed'

  arquivo_zip_full_path = source_folder + '/' + arquivo_zip

  # Verifica quantos arquivos estão no arquivo .zip
  with zipfile.ZipFile(arquivo_zip_full_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    zip_num_files = len(file_list)

  return zip_num_files


def extrai_arquivos_zip(arquivo_zip):
  source_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed'
  temp_images_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Images/Temp'
  images_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Images'

  arquivo_zip_full_path = os.path.join(source_folder, arquivo_zip)

  # Remove a pasta temporária
  if os.path.exists(temp_images_folder):
    shutil.rmtree(temp_images_folder)

  # Cria a pasta temporária novamente
  os.makedirs(temp_images_folder, exist_ok=True)

  # Extrai as imagens do arquivo zip para uma pasta temporária
  with zipfile.ZipFile(arquivo_zip_full_path, 'r') as zip_ref:
    zip_ref.extractall(temp_images_folder)

  # Conta o número de imagens extraídas
  arquivos_extraidos = os.listdir(temp_images_folder)
  num_arquivos_extraidos = len(arquivos_extraidos)

  # Verifica o número de imagens no arquivo zip
  zip_num_files = verifica_arquivos_zip(arquivo_zip)

  # Compara o número de imagens no arquivo zip com o número de imagens extraídas
  if num_arquivos_extraidos == zip_num_files:
    msg = print(f'O arquivo {arquivo_zip} foi descompactado com sucesso')
    # Move as imagens para o diretorio definitivo
    for file_name in os.listdir(temp_images_folder):
      origem = os.path.join(temp_images_folder, file_name)
      destino = os.path.join(images_folder, file_name)
      if os.path.isfile(origem):
        shutil.move(origem, destino)
    return(print(f'As imagens foram movidas para o diretório Images'))
  else:
    return(print(f'O arquivo {arquivo_zip} não foi descompactado corretamente'))



### Processa arquivos tar.gz

In [4]:
def verifica_arquivos_tar(arquivo_tar):
    source_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed'
    arquivo_tar_full_path = os.path.join(source_folder, arquivo_tar)

    # Verifica quantos arquivos estão no arquivo .tar.gz
    with tarfile.open(arquivo_tar_full_path, 'r:gz') as tar_ref:
        file_list = tar_ref.getnames()  # Lista os arquivos dentro do tar.gz
        tar_num_files = len(file_list)

    return tar_num_files


def extrai_arquivos_tar(arquivo_tar):
    source_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Compressed'
    temp_images_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Images/Temp'
    images_folder = '/content/drive/MyDrive/USP/TCC/Application/Dataset/Images'

    arquivo_tar_full_path = os.path.join(source_folder, arquivo_tar)

    # Remove a pasta temporária se ela existir
    if os.path.exists(temp_images_folder):
        shutil.rmtree(temp_images_folder)

    # Cria a pasta temporária novamente
    os.makedirs(temp_images_folder, exist_ok=True)

    # Extrai os arquivos do tar.gz para a pasta temporária
    with tarfile.open(arquivo_tar_full_path, 'r:gz') as tar_ref:
        tar_ref.extractall(temp_images_folder)

    # Conta o número de arquivos extraídos
    arquivos_extraidos = os.listdir(temp_images_folder)
    num_arquivos_extraidos = len(arquivos_extraidos)

    # Verifica o número de arquivos dentro do tar.gz
    tar_num_files = verifica_arquivos_tar(arquivo_tar)

    # Compara a contagem de arquivos extraídos e originais no tar.gz
    if num_arquivos_extraidos == tar_num_files:
        print(f'O arquivo {arquivo_tar} foi descompactado com sucesso')

        # Apaga todos os arquivos que se iniciam por ._
        for file_name in os.listdir(temp_images_folder):
            if file_name.startswith('.'):
              file_path = os.path.join(temp_images_folder, file_name)
              os.remove(file_path)

        # Move os arquivos para o diretório definitivo
        for file_name in arquivos_extraidos:
            origem = os.path.join(temp_images_folder, file_name)
            destino = os.path.join(images_folder, file_name)
            if os.path.isfile(origem):
                shutil.move(origem, destino)
        if os.path.exists(temp_images_folder):
          shutil.rmtree(temp_images_folder)

        print('As imagens foram movidas para o diretório Images')
    else:
        print(f'O arquivo {arquivo_tar} não foi descompactado corretamente')

In [5]:
arquivos_tar = ['macroscopic1.tar.gz', 'macroscopic2.tar.gz',
                'macroscopic3.tar.gz', 'macroscopic4.tar.gz']

extrai_arquivos_zip('macroscopic0.zip')

for arquivo_tar in arquivos_tar:
  extrai_arquivos_tar(arquivo_tar)

O arquivo macroscopic0.zip foi descompactado com sucesso
As imagens foram movidas para o diretório Images
O arquivo macroscopic1.tar.gz foi descompactado com sucesso
As imagens foram movidas para o diretório Images
O arquivo macroscopic2.tar.gz foi descompactado com sucesso
As imagens foram movidas para o diretório Images
O arquivo macroscopic3.tar.gz foi descompactado com sucesso
As imagens foram movidas para o diretório Images
O arquivo macroscopic4.tar.gz foi descompactado com sucesso
As imagens foram movidas para o diretório Images


---

<br>

### Analisar as imagens